# Naive Radical Polymerization Model

## Theory

Radical polymerization is a type of chain-growth polymerization in which the active center is a radical. The reaction mechanism typically involves three key steps: initiation, propagation, and termination. For a closed system with constant rate coefficients, the transient species balances are:

\begin{align*}
\frac{\textrm{d}[I]}{\textrm{d}t} & = - k_d [I] \\
\frac{\textrm{d}[M]}{\textrm{d}t} & = - k_p [M] \sum_{j=1}^{\infty}[R^{\cdot}_{j}] - 2 f k_d [I] \\
\frac{\textrm{d}[R^{\cdot}_{n}]}{\textrm{d}t} & = k_p[M]\left([R^{\cdot}_{n-1}]-[R^{\cdot}_{n}]\right)- 2 (k_{tc} + k_{td}) [R^{\cdot}_{n}] \sum_{j=1}^{\infty}[R^{\cdot}_{j}] + \delta_{n,1}2fk_d[I] \\
\frac{\textrm{d}[D_n]}{\textrm{d}t} & = k_{tc} \sum_{j=1}^{n-1}[R^{\cdot}_{n-j}][R^{\cdot}_{j}] + 2 k_{td} [R^{\cdot}_{n}] \sum_{j=1}^{\infty}[R^{\cdot}_{j}]
\end{align*}

By defining an appropriate upper limit for the chain length $N$, we obtain a closed system of $2N+2$ ordinary differential equations (ODEs). However, numerically solving this ODE system is challenging. Under typical polymerization conditions, very long chains are formed, requiring rather large $N$ values (possibly $10^4$-$10^5$ or higher). Moreover, the presence of short-lived radical species makes the ODE system stiff, requiring implicit solvers.

Special numerical methods are therefore needed to solve the population balances efficiently and stably. You can read about it [here](https://onlinelibrary.wiley.com/doi/abs/10.1002/mren.202000010). In this tutorial, we integrate the discrete population balances directly. This brute-force approach is quite inefficient (be prepared to wait 10-60 s, depending on CPU performance), but it serves a pedagogical purpose by providing transparency in its workings.

## Numerical Solution

In [1]:
import numpy as np
import plotly.graph_objects as go
from numba import njit, prange
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

If running this notebook in VS Code, please uncomment the extra lines below. They are required to render LaTeX in plotly + VS Code.

In [2]:
# import plotly
# from IPython.display import HTML, display

# plotly.offline.init_notebook_mode()
# display(HTML(
#     '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
# ))

### Model Equations

First, we implement separate functions to compute the differential mole balances for each of the three reaction steps. All functions are jitted with [numba](https://numba.pydata.org/) to improve performance (we need all the help we can!).

In [3]:
@njit(fastmath=True)
def initiation(I: float,
               M: float,
               kd: float,
               f: float
               ) -> tuple[float, float, float]:
    """Initiation balances.

    I -> 2 I*
    I* + M -> R1

    d[I]/dt = -kd*[I]
    d[M]/dt = -2*f*kd*[I]
    d[R1]/dt = 2*f*kd*[I]

    Parameters
    ----------
    I : float
        Initiator concentration.
    M : float
        Monomer concentration.
    kd : float
        Initiator decomposition rate coefficient.
    f : float
        Initiation efficiency.

    Returns
    -------
    tuple[float, float, float]
        Molar balances for I, M and R1.
    """
    Idot = -kd*I
    ri = 2*f*kd*I*(M/(M + 1e-9))
    Mdot = -ri
    R1dot = ri
    return Idot, Mdot, R1dot


@njit(fastmath=True, parallel=True)
def propagation(R: np.ndarray,
                M: float,
                kp: float
                ) -> tuple[float, np.ndarray]:
    """Propagation balances.

    R(n) + M -> R(n+1)

    d[Rn]/dt = kp*[M]*([Rn-1] - [Rn])
    d[M]/dt = -kp*[M]*sum([Rn])

    Parameters
    ----------
    R : np.ndarray
        Radical concentrations.
    M : float
        Monomer concentration.
    kp : float
        Propagation rate coefficient.

    Returns
    -------
    tuple[float, np.ndarray]
        Molar balances for M and Rn.
    """

    Mdot = -kp*M*R.sum()
    Rdot = np.zeros_like(R)
    for i in prange(1, R.size):
        Rdot[i] = kp*M*(R[i-1] - R[i])
    return Mdot, Rdot


@njit(fastmath=True, parallel=True)
def termination(R: np.ndarray,
                ktc: float,
                ktd: float
                ) -> tuple[float, np.ndarray]:
    """Termination balances.

    R(n) + R(m) -> D(n+m)
    R(n) + R(m) -> D(n) + D(m)

    d[Rn]/dt = -2*(ktc + ktd)*[Rn]*[R]
    d[Dn]/dt = ktc * sum i=1:n-1 [Rn-i]*[Ri] + 2*ktd*[Rn]*[R]

    Parameters
    ----------
    R : np.ndarray
        Radical concentrations.
    ktc : float
        Termination by combination rate coefficient.
    ktd : float
        Termination by disproportionation rate coefficient.

    Returns
    -------
    tuple[float, np.ndarray]
        Molar balances for Rn and Dn.
    """
    # d[Rn]/dt
    Rdot = -2*(ktc + ktd)*R*R.sum()
    # d[Dn]/dt
    Ddot = np.zeros_like(R)
    if ktc > 0.0:
        for n in prange(2, R.size):
            accum = 0.0
            for i in range(1, n):
                accum += R[i]*R[n-i]
            Ddot[n] = accum
        Ddot *= ktc
    Ddot += 2*ktd*R*R.sum()
    return Rdot, Ddot

Next, we combine all reaction steps in a function that calculates the total rate of change of the state vector.

In [4]:
@njit(fastmath=True, parallel=True)
def model_ydot(t: float,
               y: np.ndarray,
               kd: float,
               kp: float,
               ktc: float,
               ktd: float,
               f: float,
               N: int
               ) -> np.ndarray:
    """Calculate derivative of the state vector, dy/dt.

    y = [I, M, R0..RN, D0..DN]

    Parameters
    ----------
    t : float
        Time.
    y : np.ndarray
        State vector.
    kd : float
        Initiator decomposition rate coefficient.
    kp : float
        Propagation rate coefficient.
    ktc : float
        Termination by combination rate coefficient.
    ktd : float
        Termination by disproportionation rate coefficient.
    f : float
        Initiation efficiency.
    N : int
        Maximum chain length

    Returns
    -------
    np.ndarray
        Time derivative of the state vector.
    """
    # Unpack the state vector
    I = y[0]
    M = y[1]
    R = y[2:N+3]
    # D = y[N+3:]
    # Compute the kinetic steps
    Idot, Mdot, R1dot = initiation(I, M, kd, f)
    Mdot_, Rdot = propagation(R, M, kp)
    Mdot += Mdot_
    Rdot_, Ddot = termination(R, ktc, ktd)
    Rdot += Rdot_
    Rdot[1] += R1dot
    # Assemble derivative vector
    ydot = np.concatenate((np.array([Idot]), np.array([Mdot]), Rdot, Ddot))
    return ydot

Finally, we perform the numerical integration using a suitable ODE solver. This system is _very_ [stiff](https://en.wikipedia.org/wiki/Stiff_equation), therefore we need to chose an implicit solver like `LSODA`.

In [5]:
def solve_model(I0: float,
                M0: float,
                kd: float,
                kp: float,
                ktc: float,
                ktd: float,
                f: float,
                N: int,
                tend: float
                ) -> tuple[np.ndarray, ...]:
    """Solve the dynamic model.

    Parameters
    ----------
    I0 : float
        Initial initiator concentration.
    M0 : float
        Initial monomer concentration.
    kd : float
        Initiator decomposition rate coefficient.
    kp : float
        Propagation rate coefficient.
    ktc : float
        Termination by combination rate coefficient.
    ktd : float
        Termination by disproportionation rate coefficient.
    f : float
        Initiation efficiency.
    N : int
        Maximum chain length.
    tend : float
        Final time.

    Returns
    -------
    tuple[np.ndarray, ...]
        Time, I, M, Rn, Dn.
    """

    # Chain lengths
    s = np.arange(0, N+1)

    # Initial conditions
    R0 = np.zeros_like(s)
    D0 = np.zeros_like(s)
    y0 = np.concatenate(([I0], [M0], R0, D0))

    # Solve ODE set
    teval = np.linspace(0, tend, num=100)
    atol = np.concatenate(
        ([1e-6], [1e-6], 1e-11*np.ones_like(R0), 1e-7*np.ones_like(D0)))

    solution = solve_ivp(model_ydot,
                         t_span=(0, tend),
                         y0=y0,
                         t_eval=teval,
                         args=(kd, kp, ktc, ktd, f, N),
                         method='LSODA',  # implicit solver is a MUST
                         rtol=1e-3,
                         atol=atol)

    # Unpack the solution
    t = solution.t
    y = solution.y
    I = y[0, :]
    M = y[1, :]
    R = y[2:N+3, :]
    D = y[N+3:, :]

    return t, I, M, R, D

### Input Parameters

Let's start with some educated guesses, while being carefull not to make very long chains.

In [6]:
# Parameters
kp = 4e3  # L/(mol.s)
ktc = 1e8  # L/(mol.s)
ktd = 1e8  # L/(mol.s)
kd = 4e-4  # 1/s
f = 0.5

# Initial concentrations
M0 = 1.0  # mol/L
I0 = 1e-2  # mol/L

# Simulation time
tend = 3600  # s

# Maximum chain length
N = 1000

### Check $N$ vs Kinetic Chain Length

The initial kinetic chain length informs us about the average degree of polymerization of the macroradicals. `N` must be well above this value, otherwise chains escape the numerical domain.

In [7]:
kinetic_chain_length = kp/(2*np.sqrt(f*kd*(ktc+ktd))) * M0/np.sqrt(I0)

print(f"The kinetic chain length is {kinetic_chain_length:.0f}.")
if (N < 5*kinetic_chain_length):
    answer = "insufficient"
else:
    answer = "sufficient"
print(f"N={N} appears {answer}.")

The kinetic chain length is 100.
N=1000 appears sufficient.


### ODE Solution

This might take a while, depending on how large $N$ is. By the way, can you guess or identify which part of the code is rate-limiting?

In [8]:
t, I, M, R, D = solve_model(I0, M0, kd, kp, ktc, ktd, f, N, tend)

### Plots

#### Reactants

In [9]:
fig = make_subplots(rows=2, shared_xaxes=True)

# Add trace for [I] on the first row
fig.add_trace(go.Scatter(x=t, y=I, mode='lines', name='[I]', showlegend=False), row=1, col=1)
fig.update_yaxes(title_text=r"$[I] \text{ (mol/L)}$", row=1, col=1)

# Add trace for [M] on the second row
fig.add_trace(go.Scatter(x=t, y=M, mode='lines', name='[M]', showlegend=False), row=2, col=1)
fig.update_yaxes(title_text=r"$[M] \text{ (mol/L)}$", row=2, col=1)

# Set x-axis label (common for both subplots)
fig.update_xaxes(title_text="Time (s)", row=2, col=1)

fig.update_layout(
    title=dict(text="Reactant Concentrations", x=0.5),
    width=700, height=600)
fig.show()

#### Moments

In [10]:
def moment(P: np.ndarray, k: int) -> np.ndarray:
    "k-th moment of P"
    s = np.arange(0, P.shape[0])
    return np.dot(s**k, P)

In [11]:
fig2 = make_subplots(rows=4, cols=1, shared_xaxes=True)
eps = np.finfo(float).eps

# 0-th moment of R*
m0_R = moment(R, 0)
fig2.add_trace(go.Scatter(x=t, y=m0_R, mode='lines', name='[R*]',
                          showlegend=False), row=1, col=1)
fig2.update_yaxes(title_text=r"$[R^{\cdot}]\text{ (mol/L)}$", row=1, col=1)

# Number-average degree of polymerization of R*
m1_R = moment(R, 1)
DPn_R = m1_R / (m0_R + eps)
fig2.add_trace(go.Scatter(x=t, y=DPn_R, mode='lines', name="DPn(R*)",
                          showlegend=False), row=2, col=1)
fig2.update_yaxes(title_text=r"$DP_n(R^{\cdot})$", row=2, col=1)

# 1st moment of D
m1_D = moment(D, 1)
fig2.add_trace(go.Scatter(x=t, y=m1_D, mode='lines', name=r"l1(D)",
                          showlegend=False), row=3, col=1)
fig2.update_yaxes(title_text=r"$\lambda_1(D)$", row=3, col=1)

# Number-average degree of polymerization of D
m0_D = moment(D, 0)
DPn = m1_D / (m0_D + eps)
fig2.add_trace(go.Scatter(x=t, y=DPn, mode='lines', name="DPn(D)",
                          showlegend=False), row=4, col=1)
fig2.update_yaxes(title_text=r"$DP_n(D)$", row=4, col=1)

# Set x-axis label (common for both subplots)
fig2.update_xaxes(title_text="Time (s)", row=4, col=1)

fig2.update_layout(
    title=dict(text="Moments of R* and D", x=0.5),
    width=700, height=800)
fig2.show()

mass_balance_error = np.max(np.abs((m1_D + M)/M0 - 1.0))
print(f"Mass balance error: {mass_balance_error:.1e}")

Mass balance error: 3.2e-04


#### Chain Length Distribution

In [12]:
fig3 = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Chain length(s)
s = np.arange(0, R.shape[0])

# Normalize Rn and Dn and add traces for each
for i, Y in enumerate([R, D]):
    for ii in [1, int(0.25*len(t)), int(0.5*len(t)), int(0.75*len(t)), -1]:
        label = f"t={t[ii]:.1e}"
        y = Y[:, ii].copy()
        y /= (y.sum() + eps)
        fig3.add_trace(go.Scatter(x=s, y=y, mode='lines', name=label,
                                  showlegend=(i == 0)),  row=i+1, col=1)

fig3.update_yaxes(title_text=r"$R^{\cdot}_n$", row=1, col=1)
fig3.update_yaxes(title_text=r"$D_n$", row=2, col=1)

fig3.update_xaxes(title_text="Chain length", row=2, col=1)
fig3.update_layout(
    title=dict(text="Number Chain Length Distributions Over Time", x=0.5),
    height=800, width=700,
    legend=dict(x=1.05, y=1, orientation="v"))
fig3.show()

## Questions/Extras

* Calculate $[R^{\cdot}]$ using the QSSA approximation, and compare it to the value obtained by solving the transient radical balances.
* What is the relationship between the average degree of polymerization of $R^{\cdot}$ and $D$?
* What is the physical meaning of the moment $\lambda_1(D)$?
* What kind of distribution does $R^{\cdot}_n$ follow? And what about $D_n$?